In [ ]:
ue takes longer, I’m open to pushing to Fall 2028—but 2027 would be ideal. I want to eventually publish in top ML venues like NeurIPS, EMNLP, ICLR, ICML, etc., and I’# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.18.0


In [14]:
# The paper mentions rotation, zooming, shearing, and flipping
data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomShear(x_factor=(0.0, 0.2))  # Fixed format
  ],
  name="data_augmentation",
)

In [5]:
# Parameters from base paper
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 38
DATA_DIR = '../input/plantvillage' 

print(f"Image size set to: {IMAGE_SIZE}")
print(f"Data directory is: {DATA_DIR}")

Image size set to: (256, 256)
Data directory is: ../input/plantvillage


In [6]:

train_dir = '../input/plantvillage/PlantVillage/train' #training data
val_dir = '../input/plantvillage/PlantVillage/val'   #validation data

#making training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

#making validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,  
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Loaded {len(class_names)} classes.")
print(f"First 5 classes: {class_names[:5]}...")

Found 43444 files belonging to 38 classes.
Found 10861 files belonging to 38 classes.
Loaded 38 classes.
First 5 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy']...


In [9]:
# --- Define and Compile VGG19 ---

#Load the VGG19 base model (pre-trained on ImageNet)
base_model_vgg19 = tf.keras.applications.VGG19(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False, 
    weights='imagenet'
)

#Freezing the base model
base_model_vgg19.trainable = False

#Create our new model on top
model_vgg19 = Sequential([
    # Input layer
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    
    # Rescale pixels
    layers.Rescaling(1./255),
    
    # Re-use the same data_augmentation layer we defined
    data_augmentation,
    
    # The new VGG19 base
    base_model_vgg19,
    
    # The same classifier head
    layers.GlobalAveragePooling2D(),
    layers.Dense(NUM_CLASSES, activation='softmax') 
], name="VGG19_Transfer_Learning")

#Compile the model
#Adam optimizer and categorical cross-entropy
model_vgg19.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_vgg19.summary()

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "VGG19_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 8, 8, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,043,878 (76.46 MB)

 Trainable params: 19,494 (76.15 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [10]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=2,              
    restore_best_weights=True 
)

In [14]:
# we are running 15 epochs first, I think paper did 20 all-togehter
epochs_to_run = 15

history_vgg19 = model_vgg19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_to_run,
    callbacks=[early_stopping_callback]
)

print("VGG19 model training complete.")

Epoch 1/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 391s 288ms/step - accuracy: 0.5986 - loss: 1.6508 - val_accuracy: 0.7151 - val_loss: 1.1884
Epoch 2/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 388s 286ms/step - accuracy: 0.7026 - loss: 1.2116 - val_accuracy: 0.7553 - val_loss: 0.9692
Epoch 3/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 386s 284ms/step - accuracy: 0.7470 - loss: 1.0056 - val_accuracy: 0.7735 - val_loss: 0.8514
Epoch 4/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 387s 285ms/step - accuracy: 0.7756 - loss: 0.8819 - val_accuracy: 0.7883 - val_loss: 0.7716
Epoch 5/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 387s 285ms/step - accuracy: 0.7926 - loss: 0.8035 - val_accuracy: 0.7961 - val_loss: 0.7189
Epoch 6/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 386s 284ms/step - accuracy: 0.8035 - loss: 0.7389 - val_accuracy: 0.8014 - val_loss: 0.6799
Epoch 7/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 387s 285ms/step - accuracy: 0.8181 - loss: 0.6922 - val_accuracy: 0.8113 - val_loss: 0.6424
Epoch 8/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 386s 284ms/step - ac

In [15]:
model_vgg19.save("/kaggle/working/plant_model_vgg19.h5") #saving model to train it further afterwards..
print("Model saved successfully!") 

Model saved successfully!


In [2]:
from tensorflow.keras.models import load_model
#loading the model to countine training
model_save_path = "/kaggle/input/ai-vgg19/keras/default/1/plant_model_vgg19.h5"

print(f"Loading model from {model_save_path}...")
loaded_model_vgg19 = load_model(model_save_path)
print("Model loaded successfully.")

loaded_model_vgg19.summary()

Loading model from /kaggle/input/ai-vgg19/keras/default/1/plant_model_vgg19.h5...
Model loaded successfully.


Model: "VGG19_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 8, 8, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,043,880 (76.46 MB)

 Trainable params: 19,494 (76.15 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

 Optimizer params: 2 (12.00 B)

In [8]:
loaded_model_vgg19.evaluate(val_ds) #testing loaded model's accuracy to check on our previous training

340/340 ━━━━━━━━━━━━━━━━━━━━ 70s 207ms/step - accuracy: 0.8513 - loss: 0.4643


[0.4738672077655792, 0.850197970867157]

# Continuing the training after first 15 epochs...

In [9]:
loaded_model_vgg19.compile(
    optimizer='adam', #what ths fuck is this adam forrrrrrr..............
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [19]:
more_epochs_to_run = 7

history_continued = loaded_model_vgg19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=more_epochs_to_run 
)

Epoch 1/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 353s 257ms/step - accuracy: 0.8555 - loss: 0.5040 - val_accuracy: 0.8416 - val_loss: 0.5108
Epoch 2/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8567 - loss: 0.5015 - val_accuracy: 0.8431 - val_loss: 0.5020
Epoch 3/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8589 - loss: 0.4881 - val_accuracy: 0.8353 - val_loss: 0.5204
Epoch 4/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8585 - loss: 0.4829 - val_accuracy: 0.8404 - val_loss: 0.5026
Epoch 5/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8635 - loss: 0.4722 - val_accuracy: 0.8399 - val_loss: 0.4991
Epoch 6/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8652 - loss: 0.4646 - val_accuracy: 0.8443 - val_loss: 0.4849
Epoch 7/7
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 344s 253ms/step - accuracy: 0.8624 - loss: 0.4632 - val_accuracy: 0.8502 - val_loss: 0.4739


In [20]:
loaded_model_vgg19.save("/kaggle/working/plant_model_vgg19.h5")
print("Model saved successfully!") 

Model saved successfully!


# Fine-tunning...

In [11]:
vgg_base_layer = loaded_model_vgg19.get_layer('vgg19') 
# Unfreeze base layer for fine tunninf
vgg_base_layer.trainable = True

print("VGG16 layer within the loaded model unfrozen.")

loaded_model_vgg19.summary()

VGG16 layer within the loaded model unfrozen.


Model: "VGG19_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 8, 8, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        19,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,043,878 (76.46 MB)

 Trainable params: 20,043,878 (76.46 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
loaded_model_vgg19.compile(
    optimizer=Adam(learning_rate=1e-5), # set learning rate to 0.00001
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model re-compiled for fine-tuning.")

Model re-compiled for fine-tuning.


In [13]:
fine_tune_epochs = 3
previous_total_epochs = 20
# adding previous epochs with fine tune ones

total_epochs = previous_total_epochs + fine_tune_epochs 

history_fine_tune = loaded_model_vgg19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=previous_total_epochs  
                                        
)

Epoch 21/23
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1238s 899ms/step - accuracy: 0.8967 - loss: 0.3273 - val_accuracy: 0.8951 - val_loss: 0.3376
Epoch 22/23
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1218s 897ms/step - accuracy: 0.9601 - loss: 0.1253 - val_accuracy: 0.9607 - val_loss: 0.1239
Epoch 23/23
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1223s 900ms/step - accuracy: 0.9735 - loss: 0.0806 - val_accuracy: 0.9450 - val_loss: 0.1791


In [14]:
loaded_model_vgg19.save("/kaggle/working/plant_model_vgg19.h5")
print("Model saved successfully!") 

Model saved successfully!
